In [2]:
!pip install cryptography
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
import base64
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.exceptions import InvalidSignature
from pathlib import Path
import os
from cryptography.hazmat.backends import default_backend
import requests
import datetime

In [3]:
# Cell 1 — load KALSHI keys from .env in the same directory as this notebook


# --- Try to use python-dotenv if it's installed (recommended) ---
try:
    from dotenv import load_dotenv  # pip install python-dotenv
    env_path = Path.cwd() / ".env"   # assumes notebook is run from the same directory as .env
    if not env_path.exists():
        raise FileNotFoundError(f".env not found at: {env_path}")
    load_dotenv(dotenv_path=env_path, override=False)
except ImportError:
    # --- Fallback: minimal .env parser (no extra install needed) ---
    env_path = Path.cwd() / ".env"
    if not env_path.exists():
        raise FileNotFoundError(f".env not found at: {env_path}")

    for line in env_path.read_text().splitlines():
        line = line.strip()
        if not line or line.startswith("#") or "=" not in line:
            continue
        k, v = line.split("=", 1)
        k = k.strip()
        v = v.strip().strip('"').strip("'")
        os.environ.setdefault(k, v)  # don't overwrite if already set

# --- Read your keys into variables ---
public_key = os.getenv("KALSHI-ACCESS-KEY-DEMO")

missing = [k for k, v in {
    "KALSHI-ACCESS-KEY": public_key
}.items() if not v]

if missing:
    raise KeyError(f"Missing env var(s): {missing}. Check your .env formatting and key names.")

print("Loaded Kalshi keys ✅")  # intentionally not printing the secret values


def load_private_key_from_file(file_path):
    with open(file_path, "rb") as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None,  # or provide a password if your key is encrypted
            backend=default_backend()
        )
    return private_key


def sign_pss_text(private_key: rsa.RSAPrivateKey, text: str) -> str:
    message = text.encode('utf-8')
    try:
        signature = private_key.sign(
            message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.DIGEST_LENGTH
            ),
            hashes.SHA256()
        )
        return base64.b64encode(signature).decode('utf-8')
    except InvalidSignature as e:
        raise ValueError("RSA sign PSS failed") from e
    

def make_authenticated_request(path, method='GET', params=None):
    """Make an authenticated request to Kalshi demo API."""
    current_time = datetime.datetime.now()
    timestamp = current_time.timestamp()
    current_time_milliseconds = int(timestamp * 1000)
    timestamp_str = str(current_time_milliseconds)
    
    private_key = load_private_key_from_file('pbhaskarademo.txt')
    
    base_url = 'https://demo-api.kalshi.co'
    
    # Strip query parameters from path before signing
    path_without_query = path.split('?')[0]
    msg_string = timestamp_str + method + path_without_query
    sig = sign_pss_text(private_key, msg_string)
    
    headers = {
        'KALSHI-ACCESS-KEY': public_key,
        'KALSHI-ACCESS-SIGNATURE': sig,
        'KALSHI-ACCESS-TIMESTAMP': timestamp_str
    }
    
    if method == 'GET':
        response = requests.get(base_url + path, headers=headers, params=params)
    else:
        response = requests.post(base_url + path, headers=headers, json=params)
    
    return response

Loaded Kalshi keys ✅


In [5]:
# ============================================================================
# PULL MARKET DATA FOR KXSNOWSTORM-26JANNYC (NYC Snowstorm Jan 24-26)
# ============================================================================
# URL: https://kalshi.com/markets/kxsnowstorm/snowstorms/kxsnowstorm-26jannyc
# KXSNOWSTORM-26JANNYC is an EVENT ticker (multiple markets at snow thresholds)
# Public API — no authentication required for market data endpoints

import json
import time
from datetime import datetime, timezone

BASE_URL = "https://api.elections.kalshi.com/trade-api/v2"
EVENT_TICKER = "KXSNOWSTORM-26JANNYC"
SERIES_TICKER = "KXSNOWSTORM"

# --- 1. Get Event + All Markets ---
print("=" * 80)
print(f"FETCHING EVENT: {EVENT_TICKER}")
print("=" * 80)

event_resp = requests.get(
    f"{BASE_URL}/events/{EVENT_TICKER}",
    params={"with_nested_markets": True}
)
print(f"Status: {event_resp.status_code}")

if event_resp.status_code == 200:
    event_json = event_resp.json()
    event_data = event_json.get("event", {})

    print(f"\n--- Event Details ---")
    print(f"  Event Ticker:       {event_data.get('event_ticker')}")
    print(f"  Title:              {event_data.get('title')}")
    print(f"  Sub-title:          {event_data.get('sub_title')}")
    print(f"  Category:           {event_data.get('category')}")
    print(f"  Series Ticker:      {event_data.get('series_ticker')}")
    print(f"  Strike Date:        {event_data.get('strike_date')}")
    print(f"  Strike Period:      {event_data.get('strike_period')}")
    print(f"  Mutually Exclusive: {event_data.get('mutually_exclusive')}")

    # Get all markets in this event
    all_markets = event_data.get("markets", [])
    if not all_markets:
        # Fallback: try the deprecated top-level field
        all_markets = event_json.get("markets", [])

    print(f"\n--- Markets in this Event ({len(all_markets)}) ---")
    print(f"  {'Ticker':45s} | {'Last Price':>10s} | {'Volume':>10s} | {'Status':>10s} | {'Result':>6s}")
    print(f"  {'-'*45}-+-{'-'*10}-+-{'-'*10}-+-{'-'*10}-+-{'-'*6}")
    for m in all_markets:
        print(
            f"  {m.get('ticker','N/A'):45s} | "
            f"${m.get('last_price_dollars','N/A'):>9s} | "
            f"{m.get('volume_fp','N/A'):>10s} | "
            f"{m.get('status','N/A'):>10s} | "
            f"{m.get('result','N/A'):>6s}"
        )
else:
    print(f"Error: {event_resp.text}")
    event_data = {}
    all_markets = []

print(f"\nTotal markets: {len(all_markets)}")
print(f"Variables: event_data, all_markets")

FETCHING EVENT: KXSNOWSTORM-26JANNYC
Status: 200

--- Event Details ---
  Event Ticker:       KXSNOWSTORM-26JANNYC
  Title:              Snow in New York City from Jan 24–26?
  Sub-title:          In Jan 2026
  Category:           Climate and Weather
  Series Ticker:      KXSNOWSTORM
  Strike Date:        None
  Strike Period:      
  Mutually Exclusive: False

--- Markets in this Event (10) ---
  Ticker                                        | Last Price |     Volume |     Status | Result
  ----------------------------------------------+------------+------------+------------+-------
  KXSNOWSTORM-26JANNYC-2.0                      | $   0.9900 |   52613.00 |  finalized |    yes
  KXSNOWSTORM-26JANNYC-4.0                      | $   0.9900 |   74825.00 |  finalized |    yes
  KXSNOWSTORM-26JANNYC-6.0                      | $   0.9900 |  154721.00 |  finalized |    yes
  KXSNOWSTORM-26JANNYC-8.0                      | $   0.9900 |  314905.00 |  finalized |    yes
  KXSNOWSTORM-26JANNYC-10

In [6]:
# --- 2. Detailed View of Each Market ---
print("=" * 80)
print("DETAILED MARKET DATA")
print("=" * 80)

for i, market in enumerate(all_markets, 1):
    print(f"\n{'─'*80}")
    print(f"Market {i}/{len(all_markets)}: {market.get('ticker')}")
    print(f"{'─'*80}")
    print(f"  Title:           {market.get('title')}")
    print(f"  Status:          {market.get('status')}")
    print(f"  Result:          {market.get('result')}")
    print(f"  Market Type:     {market.get('market_type')}")
    print(f"  Open Time:       {market.get('open_time')}")
    print(f"  Close Time:      {market.get('close_time')}")
    print(f"  Yes Bid:         {market.get('yes_bid_dollars')}")
    print(f"  Yes Ask:         {market.get('yes_ask_dollars')}")
    print(f"  No Bid:          {market.get('no_bid_dollars')}")
    print(f"  No Ask:          {market.get('no_ask_dollars')}")
    print(f"  Last Price:      {market.get('last_price_dollars')}")
    print(f"  Volume:          {market.get('volume_fp')}")
    print(f"  Open Interest:   {market.get('open_interest_fp')}")
    print(f"  Liquidity ($):   {market.get('liquidity_dollars')}")
    print(f"  Settlement:      {market.get('settlement_value_dollars')}")

# Print rules from the first market (rules are typically shared across the event)
if all_markets:
    print(f"\n{'='*80}")
    print("MARKET RULES (from first market)")
    print(f"{'='*80}")
    print(f"  Primary:   {all_markets[0].get('rules_primary', 'N/A')}")
    print(f"  Secondary: {all_markets[0].get('rules_secondary', 'N/A')}")

DETAILED MARKET DATA

────────────────────────────────────────────────────────────────────────────────
Market 1/10: KXSNOWSTORM-26JANNYC-2.0
────────────────────────────────────────────────────────────────────────────────
  Title:           Snow in New York City from Jan 24–26?
  Status:          finalized
  Result:          yes
  Market Type:     binary
  Open Time:       2026-01-23T03:00:00Z
  Close Time:      2026-01-27T04:59:00Z
  Yes Bid:         0.0000
  Yes Ask:         1.0000
  No Bid:          0.0000
  No Ask:          1.0000
  Last Price:      0.9900
  Volume:          52613.00
  Open Interest:   44891.00
  Liquidity ($):   0.0000
  Settlement:      1.0000

────────────────────────────────────────────────────────────────────────────────
Market 2/10: KXSNOWSTORM-26JANNYC-4.0
────────────────────────────────────────────────────────────────────────────────
  Title:           Snow in New York City from Jan 24–26?
  Status:          finalized
  Result:          yes
  Market Type: 

In [7]:
# --- 3. Get Series Details ---
print("=" * 80)
print(f"FETCHING SERIES: {SERIES_TICKER}")
print("=" * 80)

series_resp = requests.get(
    f"{BASE_URL}/series/{SERIES_TICKER}",
    params={"include_volume": True}
)
print(f"Status: {series_resp.status_code}")

if series_resp.status_code == 200:
    series_data = series_resp.json().get("series", {})
    print(f"\n--- Series Details ---")
    print(f"  Ticker:             {series_data.get('ticker')}")
    print(f"  Title:              {series_data.get('title')}")
    print(f"  Category:           {series_data.get('category')}")
    print(f"  Frequency:          {series_data.get('frequency')}")
    print(f"  Tags:               {series_data.get('tags')}")
    print(f"  Fee Type:           {series_data.get('fee_type')}")
    print(f"  Fee Multiplier:     {series_data.get('fee_multiplier')}")
    print(f"  Total Volume:       {series_data.get('volume_fp')}")
    print(f"  Settlement Sources: {series_data.get('settlement_sources')}")
else:
    print(f"Error: {series_resp.text}")
    series_data = {}

FETCHING SERIES: KXSNOWSTORM
Status: 200

--- Series Details ---
  Ticker:             KXSNOWSTORM
  Title:              Snowstorms
  Category:           Climate and Weather
  Frequency:          one_off
  Tags:               ['Snow and rain']
  Fee Type:           quadratic
  Fee Multiplier:     1
  Total Volume:       6529611.00
  Settlement Sources: [{'name': 'the National Weather Service', 'url': 'https://www.weather.gov'}]


In [8]:
# --- 4. Get Orderbook for Each Market ---
print("=" * 80)
print(f"FETCHING ORDERBOOKS FOR ALL MARKETS")
print("=" * 80)

orderbooks = {}

for m in all_markets:
    ticker = m.get("ticker", "")
    ob_resp = requests.get(f"{BASE_URL}/markets/{ticker}/orderbook")

    if ob_resp.status_code == 200:
        ob_json = ob_resp.json()
        orderbooks[ticker] = ob_json
        ob_fp = ob_json.get("orderbook_fp", {})
        yes_levels = ob_fp.get("yes_dollars", [])
        no_levels = ob_fp.get("no_dollars", [])

        if yes_levels or no_levels:
            print(f"\n  {ticker}:")
            for level in yes_levels[:3]:
                print(f"    YES  Price: ${level[0]}  Qty: {level[1]}")
            for level in no_levels[:3]:
                print(f"    NO   Price: ${level[0]}  Qty: {level[1]}")
        else:
            print(f"  {ticker}: (empty — market settled/inactive)")
    else:
        print(f"  {ticker}: Error {ob_resp.status_code}")

    time.sleep(0.2)

print(f"\nOrderbooks fetched: {len(orderbooks)}")

FETCHING ORDERBOOKS FOR ALL MARKETS
  KXSNOWSTORM-26JANNYC-2.0: (empty — market settled/inactive)
  KXSNOWSTORM-26JANNYC-4.0: (empty — market settled/inactive)
  KXSNOWSTORM-26JANNYC-6.0: (empty — market settled/inactive)
  KXSNOWSTORM-26JANNYC-8.0: (empty — market settled/inactive)
  KXSNOWSTORM-26JANNYC-10.0: (empty — market settled/inactive)
  KXSNOWSTORM-26JANNYC-12.0: (empty — market settled/inactive)
  KXSNOWSTORM-26JANNYC-15.0: (empty — market settled/inactive)
  KXSNOWSTORM-26JANNYC-20.0: (empty — market settled/inactive)
  KXSNOWSTORM-26JANNYC-18.0: (empty — market settled/inactive)
  KXSNOWSTORM-26JANNYC-24.0: (empty — market settled/inactive)

Orderbooks fetched: 10


In [9]:
# --- 5. Get Trades for All Markets in the Event ---
print("=" * 80)
print(f"FETCHING TRADES FOR ALL MARKETS")
print("=" * 80)

all_trades = {}

for m in all_markets:
    ticker = m.get("ticker", "")
    market_trades = []
    cursor = None

    while True:
        params = {"ticker": ticker, "limit": 1000}
        if cursor:
            params["cursor"] = cursor

        trades_resp = requests.get(f"{BASE_URL}/markets/trades", params=params)

        if trades_resp.status_code != 200:
            print(f"  {ticker}: Error {trades_resp.status_code}")
            break

        data = trades_resp.json()
        trades = data.get("trades", [])
        market_trades.extend(trades)

        cursor = data.get("cursor")
        if not cursor or not trades:
            break
        time.sleep(0.3)

    all_trades[ticker] = market_trades
    
    if market_trades:
        yes_prices = [float(t["yes_price_dollars"]) for t in market_trades if t.get("yes_price_dollars")]
        total_contracts = sum(int(t.get("count", 0)) for t in market_trades)
        print(f"  {ticker}: {len(market_trades)} trades, {total_contracts:,} contracts", end="")
        if yes_prices:
            print(f", avg=${sum(yes_prices)/len(yes_prices):.4f}, range=[${min(yes_prices):.4f}, ${max(yes_prices):.4f}]")
        else:
            print()
    else:
        print(f"  {ticker}: 0 trades")

    time.sleep(0.2)

total_trade_count = sum(len(v) for v in all_trades.values())
print(f"\nTotal trades across all markets: {total_trade_count}")

FETCHING TRADES FOR ALL MARKETS
  KXSNOWSTORM-26JANNYC-2.0: 312 trades, 52,613 contracts, avg=$0.9798, range=[$0.8500, $0.9900]
  KXSNOWSTORM-26JANNYC-4.0: 842 trades, 74,825 contracts, avg=$0.9542, range=[$0.7000, $0.9900]
  KXSNOWSTORM-26JANNYC-6.0: 1673 trades, 154,721 contracts, avg=$0.9097, range=[$0.7800, $0.9900]
  KXSNOWSTORM-26JANNYC-8.0: 3989 trades, 314,905 contracts, avg=$0.8203, range=[$0.5100, $0.9900]
  KXSNOWSTORM-26JANNYC-10.0: 12752 trades, 1,266,810 contracts, avg=$0.6974, range=[$0.0100, $0.9900]
  KXSNOWSTORM-26JANNYC-12.0: 24148 trades, 2,410,880 contracts, avg=$0.3309, range=[$0.0100, $0.9900]
  KXSNOWSTORM-26JANNYC-15.0: 7717 trades, 975,270 contracts, avg=$0.1884, range=[$0.0100, $0.9400]
  KXSNOWSTORM-26JANNYC-20.0: 1175 trades, 178,224 contracts, avg=$0.0672, range=[$0.0100, $0.7500]
  KXSNOWSTORM-26JANNYC-18.0: 2237 trades, 374,044 contracts, avg=$0.0823, range=[$0.0100, $0.8700]
  KXSNOWSTORM-26JANNYC-24.0: 861 trades, 200,346 contracts, avg=$0.0390, range=

In [15]:
# --- 6. Get Candlestick Data (hourly) for Each Market ---
print("=" * 80)
print(f"FETCHING CANDLESTICKS FOR ALL MARKETS")
print("=" * 80)

candle_data = {}

for m in all_markets:
    ticker = m.get("ticker", "")
    open_time_str = m.get("open_time", "")
    close_time_str = m.get("close_time", "")

    if not open_time_str or not close_time_str:
        print(f"  {ticker}: No open/close time — skipping")
        continue

    open_dt = datetime.fromisoformat(open_time_str.replace("Z", "+00:00"))
    close_dt = datetime.fromisoformat(close_time_str.replace("Z", "+00:00"))
    start_ts = int(open_dt.timestamp())
    end_ts = int(close_dt.timestamp())

    candle_resp = requests.get(
        f"{BASE_URL}/series/{SERIES_TICKER}/markets/{ticker}/candlesticks",
        params={
            "start_ts": start_ts,
            "end_ts": end_ts,
            "period_interval": 60  # hourly
        }
    )

    if candle_resp.status_code == 200:
        cdata = candle_resp.json()
        sticks = cdata.get("candlesticks", [])
        candle_data[ticker] = sticks
        print(f"  {ticker}: {len(sticks)} hourly candlesticks")
    else:
        print(f"  {ticker}: Error {candle_resp.status_code}")
        candle_data[ticker] = []

    time.sleep(0.3)

total_candles = sum(len(v) for v in candle_data.values())
print(f"\nTotal candlesticks across all markets: {total_candles}")

FETCHING CANDLESTICKS FOR ALL MARKETS
  KXSNOWSTORM-26JANNYC-2.0: 92 hourly candlesticks
  KXSNOWSTORM-26JANNYC-4.0: 83 hourly candlesticks
  KXSNOWSTORM-26JANNYC-6.0: 91 hourly candlesticks
  KXSNOWSTORM-26JANNYC-8.0: 95 hourly candlesticks
  KXSNOWSTORM-26JANNYC-10.0: 97 hourly candlesticks
  KXSNOWSTORM-26JANNYC-12.0: 97 hourly candlesticks
  KXSNOWSTORM-26JANNYC-15.0: 97 hourly candlesticks
  KXSNOWSTORM-26JANNYC-20.0: 89 hourly candlesticks
  KXSNOWSTORM-26JANNYC-18.0: 90 hourly candlesticks
  KXSNOWSTORM-26JANNYC-24.0: 90 hourly candlesticks

Total candlesticks across all markets: 921


In [ ]:
# --- 7. Summary ---
print("=" * 80)
print("DATA SUMMARY")
print("=" * 80)
print(f"\n  Event Ticker:    {EVENT_TICKER}")
print(f"  Series Ticker:   {SERIES_TICKER}")
print(f"  Event Title:     {event_data.get('title', 'N/A')}")
print(f"  Total Markets:   {len(all_markets)}")
print(f"  Total Trades:    {sum(len(v) for v in all_trades.values())}")
print(f"  Total Candles:   {sum(len(v) for v in candle_data.values())}")

print(f"\n  Variables available for further analysis:")
print(f"    event_data     — dict with event details")
print(f"    all_markets    — list of all market dicts in this event")
print(f"    series_data    — dict with series metadata")
print(f"    orderbooks     — dict[ticker] -> orderbook data")
print(f"    all_trades     — dict[ticker] -> list of trade records")
print(f"    candle_data    — dict[ticker] -> list of hourly candlesticks")
print("=" * 80)